In [4]:
%load_ext autoreload
%autoreload 2

from osu.vae import OsuReplayVAE
from osu.rnn import OsuReplayRNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
RNN_PATH = ".trained/rnn_most_recent.pt"
rnn = OsuReplayRNN.load(RNN_PATH)

Position model compiled with torch.compile
RNN Model initialized on cuda (noise_std=0.0)
Position model parameters: 227650
RNN model loaded from .trained/rnn_most_recent.pt


In [5]:
VAE_PATH = ".trained/vae_most_recent.pt"
vae = OsuReplayVAE.load(VAE_PATH)

VAE models compiled..
VAE Models initialized on cuda
Encoder parameters: 236864
Decoder parameters: 260418
VAE models loaded from .trained/vae_most_recent.pt


In [6]:
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.dataset as dataset
import torch
import numpy as np

test_name = 'ringyou'
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'
test_mods = 0

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

Turning beatmaps into time series data: 100%|██████████| 1/1 [00:00<00:00,  3.96it/s]


In [7]:
model = vae

# generate 10 plays
NUM_PLAYS = 10

# get an array of [[x,y]] for each play
plays = [np.concatenate(model.generate(data)) for _ in range(NUM_PLAYS)]

In [10]:
# get the mean absolute error for two plays
def get_mae(play1, play2):
    return np.mean(np.abs(play1 - play2))

avg_mae = np.average([get_mae(plays[i], plays[i+1]) for i in range(NUM_PLAYS-1)])

In [11]:
avg_mae

np.float32(0.020709597)

In [14]:
# preview a replay
from osu.preview.preview import preview_replay_raw

replay_data = np.pad(plays[0], ((0, 0), (0, 2)), mode='constant', constant_values=0)

preview_replay_raw(replay_data, test_map_path, test_mods, test_song)

In [2]:
# analyze stuff about the data
import numpy as np

xs = np.load(f'.datasets/xs_5000_07-21_23-28-07.npy')
ys = np.load(f'.datasets/ys_5000_07-21_23-28-07.npy')